# Crash investigation and solution for explicit solvent simulations 

The nanoparticles formed in implicit solvent explode (forces diverge) during minimisation with explicit solvent. This notebook investigates why, and provides the method the setup the simulations in a stable way.

The solution is given in Fix 2, which uses a double minimization procedure:
* The nanoparticle is minimized in a vacuum.
* This structure is then solvated and neutralized in explicit water
* The minimized solvent-nanoparticle system is then thermalised as detailed below.

The structures called `Thermalized.pdb` will be used to start the production simulations.

## Hypothesis 1. 
### The initial energy of nanoparticle in explicit solvent is NaN
Testing the initial energy of the initial structure of the explicit solvent simulations. Creating the system and solvating the nanoparticle in a box of water.

In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
import time
from openmoltools.forcefield_generators import gaffTemplateGenerator

The parameter determines whether the scripts in this notebook will be run.

In [2]:
run = False

In [3]:
forcefield = ForceField('gaff.xml','tip3p.xml','amber99sbildn.xml')
forcefield.registerTemplateGenerator(gaffTemplateGenerator)

if run == True:
    ligs = ['1e','2m','3t','4r','5s','6s','7t','8t','9t','10v']

    for l in ligs:
        pdb = PDBFile(l+'/nanoparticle.pdb')
        modeller = Modeller(pdb.topology, pdb.positions)
        modeller.addSolvent(forcefield,padding=1*nanometer,neutralize=True,ionicStrength=0.0001*molar)
    
        system = forcefield.createSystem(modeller.topology,nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
        integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
    
        context = Context(system, integrator)
        context.setPositions(modeller.positions)
        context.setVelocitiesToTemperature(300*kelvin)
    
        state = context.getState(getEnergy=True)
        pot_energy = state.getPotentialEnergy()
        kin_energy = state.getKineticEnergy()
    
        print l, pot_energy,kin_energy

None of the nanoparticles in water has initial NaN energy. So why do they blow up? And what can I do about it? Run Langevin with a tiny timestep?

## Fix 1. Run GHMC at a very low temperture with a small timestep

In [4]:
from openmmtools.integrators import GHMCIntegrator

In [5]:
ligs = ['1e','2m','3t','4r','5s','6s','7t','8t','9t','10v']

if run == True:
    for l in ligs:
        print 'Running {0}'.format(l)
        pdb = PDBFile(l+'/miminised.pdb')
        modeller = Modeller(pdb.topology, pdb.positions)
        modeller.addSolvent(forcefield,padding=1*nanometer,neutralize=True,ionicStrength=0.0001*molar)
    
        system = forcefield.createSystem(modeller.topology,nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
        integrator = LangevinIntegrator(1*kelvin, 1/picosecond, 0.0001*picoseconds)
    
        context = Context(system, integrator)
        context.setPositions(modeller.positions)
        context.setVelocitiesToTemperature(10*kelvin)
    
        print '    GHMC'
        for i in range(10):
            integrator.step(100)
            state = context.getState(getEnergy=True)
            pot_energy = state.getPotentialEnergy()
            kin_energy = state.getKineticEnergy()
            print 'Energies:', pot_energy,kin_energy
        
            filename = '{0}/{1}_Langevin_1K.pdb'.format(l,i)
            positions = context.getState(getPositions=True).getPositions(asNumpy=True)
            PDBFile.writeFile(modeller.topology,positions, open(filename, 'w'))

Most break when the number of steps is greater than 50.

Trying a different protocol where I gradually heat up the system with a 0.1 fs timestep

In [6]:
ligs = ['1e','2m','3t','4r','5s','6s','7t','8t','9t','10v']

if run==True:
    forcefield = ForceField('gaff.xml','tip3p.xml','amber99sbildn.xml')
    forcefield.registerTemplateGenerator(gaffTemplateGenerator)

    for l in ligs:
        print '\n Running {0}'.format(l)
        pdb = PDBFile(l+'/miminised.pdb')
        modeller = Modeller(pdb.topology, pdb.positions)
        modeller.addSolvent(forcefield,padding=1*nanometer,neutralize=True,ionicStrength=0.0001*molar)
    
        system = forcefield.createSystem(modeller.topology,nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
        integrator = LangevinIntegrator(1*kelvin, 1/picosecond, 0.01*femtosecond)
                    
        context = Context(system, integrator)
        context.setPositions(modeller.positions)
        context.setVelocitiesToTemperature(1*kelvin) 
        positions = context.getState(getPositions=True).getPositions(asNumpy=True)
        pdbfile = open('{0}/Heated_Langevin.pdb'.format(l), 'w')
        PDBFile.writeModel(modeller.topology,positions,file=pdbfile,modelIndex=1)
    
        temp = range(1,300,9)
        Nsteps= range(1,1100,1000/(len(temp)-1))
        Nsteps.reverse()
    
        print '    Langevin gradual heating'
        for i in range(len(temp)):
            integrator.setTemperature(temp[i]*kelvin)
            integrator.step(Nsteps[i])
            print "    Completed steps at {0}K".format(temp[i])     
            positions = context.getState(getPositions=True).getPositions(asNumpy=True)
            PDBFile.writeModel(modeller.topology,positions,file=pdbfile,modelIndex=i+2)
            del integrator, context

Similuations still explode. New hypothesis needed.

## Hypothesis 2: 
### The interactions of the nanoparticle in a vaccuum are suffucient to blow the simulation up

* Ran `run_minimise.py` using `MinimiseTest.sh`
* Minimising the nanoparticles in vacuum, with no solvent or dialectric works fine, without NaNs
    - PDB outputs called `miminised.pdb`: compressed w.r.t original nanoparticle due to vacuum calculation.
    
This suggests I use the new `miminised.pdb` to seed the simulations.    

## Fix 2: Minimise the minimized nanoparticle in water
The below works, so now I have properly minimized nanoparticles in water. Hopefully the thermalised runs work.

In [7]:
if run==True:
    ligs = ['1e','2m','3t','4r','5s','6s','7t','8t','9t','10v']
    for l in ligs:
        print 'Running {0}'.format(l)
        pdb = PDBFile(l+'/miminised.pdb')
        modeller = Modeller(pdb.topology, pdb.positions)
        modeller.addSolvent(forcefield,padding=1*nanometer,neutralize=True,ionicStrength=0.0001*molar)
    
        system = forcefield.createSystem(modeller.topology,nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
        integrator = LangevinIntegrator(1*kelvin, 1/picosecond, 0.0001*picoseconds)
    
        platform = Platform.getPlatformByName('CPU')
        simulation = Simulation(modeller.topology, system, integrator, platform)
        simulation.context.setPositions(modeller.positions)    

        simulation.minimizeEnergy()
    
        filename = '{0}/DoubleMinimized.pdb'.format(l)
        positions = simulation.context.getState(getPositions=True).getPositions(asNumpy=True)
        PDBFile.writeFile(modeller.topology,positions, open(filename, 'w'))
        del system, integrator, simulation

Now seeing if I can relode the doubly minimized nanoparticle in water and thermalize.

In [8]:
if run==True:
    for l in ligs:
        print 'Running {0}'.format(l)
        pdb = PDBFile('{0}/DoubleMinimized.pdb'.format(l))
        system = forcefield.createSystem(pdb.topology,nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
        integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 2*femtoseconds)

        platform = Platform.getPlatformByName('CPU')
        simulation = Simulation(pdb.topology, system, integrator, platform)
        simulation.context.setPositions(pdb.positions)  

        # Minimizing one more time to fix imprecision in PDB file.
        simulation.minimizeEnergy()

        # Thermalizing
        simulation.context.setVelocitiesToTemperature(300*kelvin)
        simulation.step(500)

        filename = '{0}/Thermalized.pdb'.format(l)
        positions = simulation.context.getState(getPositions=True).getPositions(asNumpy=True)
        PDBFile.writeFile(pdb.topology,positions, open(filename, 'w'))

Using 'Thermalized.pdb' to seed my production runs.